In [739]:
import numpy as np
from tqdm import tqdm
from time import sleep
import operator
import copy

In [740]:
class NN():

    #creates a 3 layer NN with 2 input neuron, n hidden neuron and 1 ouput neuron
    def __init__(self, hiddenLayer_neuron = 2, pop_size = 2):

        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.hl_neuron = hiddenLayer_neuron

        self.pop_size = pop_size
        
        self.w_hidden = [np.random.uniform(size=(2,self.hl_neuron)) for _ in range(pop_size)]
        self.w_out = [np.random.uniform(size=(self.hl_neuron,1)) for _ in range(pop_size)]

        self.b_hidden = [np.random.uniform(size=(1,self.hl_neuron)) for _ in range(pop_size)]
        self.b_out = [np.random.uniform(size=(1,1)) for _ in range(pop_size)]

        self.z0s = []
        self.z1s = []

        print("Init: ")
        self.print_weights()

    #ReLU function
    def ReLU(self,x):
        return x * (x > 0) 
    
    #binary cross entropy loss function
    def bcel(self, y, y_hat):
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))

    #sigmoid function
    def sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    #Execute the NN with input x
    def execute(self,x):
        result = self.sigmoid(self.forward_pass(0,x)[0])
        return np.round(result[0])
    
    #forward pass
    def forward_pass(self,index,x):
        x = np.array(x)
        z0 = np.dot(x,self.w_hidden[index]) + self.b_hidden[index]
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden,self.w_out[index]) + self.b_out[index]
        return z1, z0

    def neuro_train(self):
        
        epoch = 1
        while self.accuracy(0)[0] != 1:
            
            print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))

            copy_w_out = self.copy(self.w_out)
            copy_b_out = self.copy(self.b_out)
            copy_w_hidden = self.copy(self.w_hidden)
            copy_b_hidden = self.copy(self.b_hidden)
            

            self.mutation(copy_w_out)
            self.mutation(copy_b_out)
            self.mutation(copy_w_hidden)
            self.mutation(copy_b_hidden)
            
            self.w_out += copy_w_out
            self.w_hidden += copy_w_hidden
            self.b_out += copy_b_out
            self.b_hidden += copy_b_hidden

            f_list = self.fitness()
            self.selection(f_list)

            epoch += 1

        print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))
        self.print_weights()
    
    def copy(self, weight):
        new_weight = []
        for w in weight:
            new_weight.append(w.copy())
        return new_weight
    
    def fitness(self):
        f_list = []
        for i in range(len(self.w_out)-1):
            acc, loss = self.accuracy(i)
            f_list += [1-acc+loss]
        return f_list

    def selection(self, f_list):

        dic = dict()
    
        for i in range(len(f_list)):
            dic.update([(i,f_list[i])])
        
        sortedPop = sorted(dic.items(),key=operator.itemgetter(1),reverse=False)
        last = sortedPop[self.pop_size:]
        last.sort()
        c = 0
        for index in last:
        
            del self.w_out[index[0]-c]
            del self.w_hidden[index[0]-c]
            del self.b_out[index[0]-c]
            del self.b_hidden[index[0]-c]
            c+=1
        

    def crossover(self, weight):
        w = np.zeros(shape=(len(weight),weight[0].shape[0],weight[0].shape[1]))
        lis = [i for i in range(len(weight))]
        np.random.shuffle(lis)
        for k in range(len(weight) // 2 - 1):
            for i in range(weight[0].shape[0]):
                for j in range(weight[0].shape[1]):
                    if np.random.randint(0,2):
                        w[lis[2*k]][i][j] = weight[lis[2*k+1]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k]][i][j]
                    else:
                        w[lis[2*k]][i][j] = weight[lis[2*k]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k+1]][i][j]
        return w

    def mutation(self, weight):
        for k in range(len(weight)):
            for i in range(len(weight[k])):
                for j in range(len(weight[k][i])):
                    weight[k][i][j] += np.random.normal(0,0.1)


    #prints the weights of the nn  
    def print_weights(self):
        print("W_hidden: " + str(self.w_hidden[0])) 
        print("B_hidden: " + str(self.b_hidden[0])) 
        print("W_out: " + str(self.w_out[0])) 
        print("B_out: " + str(self.b_out[0]), end="\n\n")
        pass

    #calculates the accuracy and current loss of the nn
    def accuracy(self, index):

        counter = 0
    
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        
        loss = 0
        for i in range(len(self.data)):
            loss += self.bcel(self.target[i],self.sigmoid(self.forward_pass(index,self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
    

In [741]:
nn = NN(4, pop_size= 10)

Init: 
W_hidden: [[0.83661433 0.71667495 0.95791531 0.44639745]
 [0.15551871 0.79080203 0.15071741 0.38661481]]
B_hidden: [[0.93735487 0.90298622 0.58943987 0.35274905]]
W_out: [[0.45968963]
 [0.34077452]
 [0.21285044]
 [0.06623959]]
B_out: [[0.25572939]]



In [742]:
nn.neuro_train()

Epoch: 1 current acc: 0.5
Epoch: 2 current acc: 0.5
Epoch: 3 current acc: 0.5
Epoch: 4 current acc: 0.5
Epoch: 5 current acc: 0.5
Epoch: 6 current acc: 0.5
Epoch: 7 current acc: 0.5
Epoch: 8 current acc: 0.5
Epoch: 9 current acc: 0.5
Epoch: 10 current acc: 0.5
Epoch: 11 current acc: 0.5
Epoch: 12 current acc: 0.5
Epoch: 13 current acc: 0.5
Epoch: 14 current acc: 0.5
Epoch: 15 current acc: 0.5
Epoch: 16 current acc: 0.5
Epoch: 17 current acc: 0.5
Epoch: 18 current acc: 0.5
Epoch: 19 current acc: 0.5
Epoch: 20 current acc: 0.5
Epoch: 21 current acc: 0.5
Epoch: 22 current acc: 0.5
Epoch: 23 current acc: 0.5
Epoch: 24 current acc: 0.5
Epoch: 25 current acc: 0.5
Epoch: 26 current acc: 0.5
Epoch: 27 current acc: 0.5
Epoch: 28 current acc: 0.5
Epoch: 29 current acc: 0.5
Epoch: 30 current acc: 0.5
Epoch: 31 current acc: 0.5
Epoch: 32 current acc: 0.5
Epoch: 33 current acc: 0.5
Epoch: 34 current acc: 0.5
Epoch: 35 current acc: 0.5
Epoch: 36 current acc: 0.5
Epoch: 37 current acc: 0.5
Epoch: 38 

In [743]:
nn.execute([0,0])

array([0.])

# Aufgabe 3


In [744]:
import tensorflow as tf

In [745]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2, 1)),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [746]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

model.compile(loss=loss_fn,
              metrics=['accuracy'])

In [747]:
data = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

target = np.array([[0],[1],[1],[0]])

In [748]:
model.fit(data,target,epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 436ms/step - loss: 0.7248 - accuracy: 0.5000
Epoch 2/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7217 - accuracy: 0.2500
Epoch 3/200
1/1 [==============================] - 0s 6ms/step - loss: 0.7195 - accuracy: 0.2500
Epoch 4/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7177 - accuracy: 0.2500
Epoch 5/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7162 - accuracy: 0.2500
Epoch 6/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7148 - accuracy: 0.2500
Epoch 7/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7135 - accuracy: 0.2500
Epoch 8/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7122 - accuracy: 0.2500
Epoch 9/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7111 - accuracy: 0.2500
Epoch 10/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7100 - accuracy: 0.2500
Epoch 11/200
1/1 [=========

In [749]:
model.evaluate(data,  target, verbose=2)

1/1 - 0s - loss: 0.6011 - accuracy: 1.0000 - 109ms/epoch - 109ms/step


[0.6011431813240051, 1.0]

In [750]:
l = [1,2,3]
l[:2]



[1, 2]